生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

读取数据集

在这里不再使用前面自行定义的函数，而是通过调用框架中现有的API来读取数据

data_arrays：
包含特征和标签的元组或列表
使用星号(*)操作符将元组解包传递给TensorDataset构造函数

batch_size：每个小批量包含的样本数量

is_train：控制是否随机打乱数据，通常在训练时设为True，在测试时设为False

In [4]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)  #创建一个张量数据集
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[-0.4431, -1.0020],
         [-1.7355,  0.0939],
         [-0.1067, -0.3002],
         [ 0.0175,  0.9006],
         [ 1.0315,  2.4028],
         [-1.5887, -0.2785],
         [ 0.4334,  0.8101],
         [ 0.4836,  1.0552],
         [-0.5107, -0.1673],
         [ 0.7220,  0.3632]]),
 tensor([[ 6.7194],
         [ 0.3964],
         [ 5.0109],
         [ 1.1711],
         [-1.8942],
         [ 1.9797],
         [ 2.3083],
         [ 1.5681],
         [ 3.7613],
         [ 4.4025]])]

对于标准深度学习模型，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。 
我们首先定义一个模型变量net，它是一个Sequential类的实例。 Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 

在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。 但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。
同时这一单层称为全连接层

In [5]:
# nn是神经网络的缩写
# 第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

现在我们能直接访问参数以设定它们的初始值。 我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。

在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零

In [6]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

定义损失函数
这里同样使用MSE，也称为平方$L_2$范数

In [9]:
loss = nn.MSELoss()

定义优化算法

小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。 当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值，这里设置为0.03。

In [8]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练

之所以要进行梯度清零，是因为PyTorch中的梯度是累加的，而不是覆盖的。如果不清零，每次调用backward()时：
    新计算的梯度会被添加到已存在的梯度上
    而不是替换原有梯度

从而导致：
    每个小批量的梯度被错误地累加
    参数更新时使用的是多个批次的梯度之和
    实际效果相当于使用了更大的批量大小，但计算方式不正确

同时也可以保持每次迭代的独立性：
    每个小批量应该是一个独立的样本集：
    每次迭代应该只基于当前批次的数据来计算梯度
    通过清零确保每次迭代的起点相同

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)  #计算预测值和损失
        trainer.zero_grad()  #清空梯度
        l.backward()    #反向传播计算梯度
        trainer.step()  #更新参数
    l = loss(net(features), labels) #计算损失
    print(f'epoch {epoch + 1}, loss {l:f}') 

epoch 1, loss 0.000264
epoch 2, loss 0.000101
epoch 3, loss 0.000100


同样进行比较

In [11]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0002, 0.0001])
b的估计误差： tensor([-7.8678e-05])
